In [2]:
import pandas as pd 
import numpy as np 
from src.utils.preprocess import get_videoid

import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats("retina")

import yaml
with open("../../src/configs.yml", "r") as configs:
    configs = yaml.safe_load(configs)

ROOTPATH = configs["ROOTPATH"]
DATAPATH = configs["DATAPATH"]
ROOTPATH2 = configs["ROOTPATH2"]
OUTPATH = "../../output/results-summary-120424/"

START = configs["START"]
END = configs["END"]
print(configs)
print(START)
print(END)

{'ROOTPATH': '/Users/yijingch/Documents/GITHUB/invisible-public/', 'ROOTPATH2': '/Users/yijingch/Documents/GITHUB/dd-full/', 'DATAPATH': '/Users/yijingch/Documents/GITHUB/dd-full/data/', 'DATAPATH2': '/Users/yijingch/Documents/GITHUB/collect-youtube-content/data/', 'INDEXPATH': '/Users/yijingch/Documents/GITHUB/dd-full/index/', 'START': datetime.datetime(2018, 5, 1, 0, 0), 'END': datetime.datetime(2023, 5, 1, 0, 0)}
2018-05-01 00:00:00
2023-05-01 00:00:00


In [3]:
view_all = pd.read_csv(DATAPATH + "youtube/yt_watch.csv", sep=";")

In [7]:
view_all[view_all["respondent_id"]==1072184]

,Unnamed: 0,respondent_id,titleUrl,channel_url,video_id,timestamp
4613764,5437703,1072184,https://www.youtube.com/watch?v=k-_s7nW8BB0,NaN,k-_s7nW8BB0,2020-09-23 17:55:48


In [2]:
view_all = pd.read_csv(DATAPATH + "youtube/yt_watch.csv", sep=";")
view_all = view_all[["respondent_id","titleUrl","channel_url", "timestamp"]].rename(columns={"titleUrl":"video_url"})
view_all["channel_id"] = view_all["channel_url"].map(lambda x: x.split("/")[-1] if str(x)!= "nan" else np.NaN)
view_all["video_id"] = view_all["video_url"].map(lambda x: get_videoid(x))
view_all = view_all.dropna(subset = ["respondent_id", "video_id", "channel_id"])[["respondent_id","channel_id","video_id","timestamp"]]

subscribe_all = pd.read_csv(DATAPATH + "youtube/yt_subscriptions.csv", sep=";")[["respondent_id","channel_id"]]

comment_all = pd.read_csv(DATAPATH + "youtube/yt_comments_from2018may_to2023may_new.csv", sep=";").rename(columns={"snippet.channelId":"channel_id"})
comment_all = comment_all[["respondent_id","video_id", "channel_id"]].dropna()
comment_public = pd.read_pickle(DATAPATH + "youtube/yt_comments_public_061824.pkl")

view_all["timestamp"] = pd.to_datetime(view_all["timestamp"])
view_all = view_all[(view_all["timestamp"]>=START)&(view_all["timestamp"]<=END)].copy()

comment_public["timestamp"] = pd.to_datetime(comment_public["published_at"]).dt.tz_localize(None)
comment_public = comment_public[(comment_public["timestamp"]>=START)&(comment_public["timestamp"]<=END)].copy()
# subscribing data cannot be filtered; sample comment is already filtered

In [3]:
pol_df = pd.read_csv(DATAPATH + "survey/pol_survey_cleaned_NEW_INCLUDEALL.csv").rename(columns={"user_id":"respondent_id"})

# get new weights [new!]
pol_df.drop(columns=["weight"], inplace=True)
new_weights = pd.read_spss(DATAPATH + "survey/TK_DataDonation_eng.sav")[["respondent_id","weight"]]
new_weights["respondent_id"] = new_weights["respondent_id"].map(lambda x: int(x))
pol_df = pol_df.merge(new_weights, on="respondent_id", how="left")

print(pol_df.columns)
set1 = set(view_all["respondent_id"].unique()).union(set(subscribe_all["respondent_id"].unique())).union(set(comment_all["respondent_id"].unique()))
set2 = set1.intersection(set(pol_df["respondent_id"].unique()))
print(len(pol_df))

Index(['respondent_id', 'gender', 'birth_year', 'education', 'fb_freq',
       'ig_freq', 'tw_freq', 'yt_freq', 'lk_freq', 'tt_freq', 'sp_freq',
       'sns_view_politics', 'sns_view_politics_friends',
       'sns_view_politics_disagree', 'sns_view_politics_progov',
       'sns_view_politics_antigov', 'seek_politics_tv', 'seek_politics_radio',
       'seek_politics_newpp', 'seek_politics_sites', 'seek_politics_sns',
       'seek_politics_convo', 'talk_politics_frfa', 'align_politics_frfa',
       'interest_politics', 'vote_party_2022', 'would_go_vote_sunday',
       'would_vote_party_sunday', 'know_sppt_fdsz', 'know_sppt_mszp',
       'know_sppt_jbbk', 'know_sppt_lmp', 'know_sppt_dk', 'know_sppt_mtum',
       'know_sppt_mhzk', 'know_relate_fdsz', 'know_relate_mszp',
       'know_relate_jbbk', 'know_relate_lmp', 'know_relate_dk',
       'know_relate_mtum', 'know_relate_mhzk', 'feel_thm_fdsz',
       'feel_thm_mszp', 'feel_thm_jbbk', 'feel_thm_lmp', 'feel_thm_dk',
       'feel_thm_mtum',

In [4]:
channel_labels_all = pd.read_excel(ROOTPATH2 + "index/yt_polclass_20240627.xlsx")
channel_labels_all

label_col = "classification_lvl1_4cat"
LABEL_MAP = {
    1: 0, # neutral 
    2: -1, # anti-gov 
    3: 1, # pro-gov
}

channel_labels = channel_labels_all[channel_labels_all[label_col].isin(LABEL_MAP.keys())]

CHANNEL_LABEL_MAP = {}
for lab,val in LABEL_MAP.items():
    channel_subdf = channel_labels[channel_labels[label_col]==lab]
    these_channels = set(channel_subdf["channel_id"].tolist())
    print(lab, val, "# of channels", len(these_channels))
    for this_ch in these_channels:
        CHANNEL_LABEL_MAP[this_ch] = val 

CHANNEL_LABEL_MAP["UCKY8HrMpm32WAMK0lauLAZA"] = {"before":-1, "after":1} 

1 0 # of channels 149
2 -1 # of channels 276
3 1 # of channels 139


In [5]:
these_channels = CHANNEL_LABEL_MAP.keys()
pol_view = view_all[view_all["channel_id"].isin(these_channels)].copy()
pol_subscribe = subscribe_all[subscribe_all["channel_id"].isin(these_channels)].copy()
pol_comment = comment_all[comment_all["channel_id"].isin(these_channels)].copy()
pol_comment_public = comment_public[comment_public["channel_id"].isin(these_channels)].copy()

In [6]:
aggr_func1 = {"channel_id_time": lambda x: list(x)}
aggr_func2 = {"channel_id":lambda x: list(x)}
pol_view["channel_id_time"] = pol_view.apply(lambda x: (x["channel_id"], x["timestamp"]), axis=1)
view_aggr = pol_view.groupby("respondent_id").agg(aggr_func1).reset_index()
subscribe_aggr = pol_subscribe.groupby("respondent_id").agg(aggr_func2).reset_index()
comment_aggr = pol_comment.groupby("respondent_id").agg(aggr_func2).reset_index()

In [7]:
view_aggr 

,respondent_id,channel_id_time
0,20145,"[(UCZsebJUpdb-cQ8RLwmDAVcg, 2019-01-24 12:18:0..."
1,20377,"[(UCSSTPAdzn12PF8XZhJJXB2w, 2018-05-09 12:38:0..."
2,21712,"[(UCw0oJgCjz3NvAICbqWImH-A, 2018-05-08 13:54:4..."
3,23815,"[(UCZXc2s97ubN4BLOV9WLZWcQ, 2018-08-25 21:29:1..."
4,26713,"[(UCcWzNTBDXJ6jpZfJYy3PH4Q, 2019-12-17 15:58:3..."
...,...,...
635,1266549,"[(UClUSqujSdARUgqlgX-FfAZA, 2022-12-30 23:39:3..."
636,1266727,"[(UCBFovo0_BUOuI1mgWSvEfqA, 2018-08-07 18:57:3..."
637,1267123,"[(UCXxUjsYxbL1llIe9Ra6wcyQ, 2020-01-03 18:38:5..."
638,1267313,"[(UCWALLPCVkSyKLuZLg5bc3QQ, 2018-10-19 13:28:4..."


In [8]:
def get_channel_label(channel_id, label_map, before_after=""):
    if before_after == "":
        return label_map[channel_id]
    else:
        return label_map[channel_id][before_after]

def count_channel_label(channel_ls, label_map=CHANNEL_LABEL_MAP):
    label_ls = []
    for c in channel_ls:
        if c in label_map.keys():
            if c == "UCKY8HrMpm32WAMK0lauLAZA":
                lab = get_channel_label(c, label_map=label_map, before_after="after")
            else:
                lab = get_channel_label(c, label_map=label_map)
            label_ls.append(lab)
    label_counter = {-1:0, 0:0, 1:0}
    for l in label_ls:
        label_counter[l] += 1
    return label_counter

def count_channel_label_temporal(channel_time_ls, label_map=CHANNEL_LABEL_MAP):
    label_ls = []
    for c,time in channel_time_ls:
        if c in label_map.keys():
            if c == "UCKY8HrMpm32WAMK0lauLAZA":
                if time <= pd.to_datetime("2020-08-01 00:00:00"):
                    lab = get_channel_label(c, label_map=label_map, before_after="before")
                else:
                    lab = get_channel_label(c, label_map=label_map, before_after="after")
            else:
                lab = get_channel_label(c, label_map=label_map)
            label_ls.append(lab)
    label_counter = {-1:0, 0:0, 1:0}
    for l in label_ls:
        label_counter[l] += 1
    return label_counter


In [9]:
view_aggr["view_labs"] = view_aggr["channel_id_time"].map(lambda x: count_channel_label_temporal(x))
subscribe_aggr["subscribe_labs"] = subscribe_aggr["channel_id"].map(lambda x: count_channel_label(x))
comment_aggr["comment_labs"] = comment_aggr["channel_id"].map(lambda x: count_channel_label(x))


In [ ]:
cols = ["respondent_id", "weight", "gender", "age", "interest_politics", "education", "feel_thm_fdsz", "lftRght"]
reg_table = pol_df[cols].merge(view_aggr, on="respondent_id", how="left")
reg_table = reg_table.merge(subscribe_aggr, on="respondent_id", how="left")
reg_table = reg_table.merge(comment_aggr, on="respondent_id", how="left")

for activity in ["view", "subscribe", "comment"]:
    reg_table[f"{activity}_anti"] = reg_table[f"{activity}_labs"].map(lambda x:x[-1] if str(x)!="nan" else 0)
    reg_table[f"{activity}_netr"] = reg_table[f"{activity}_labs"].map(lambda x:x[0] if str(x)!="nan" else 0)
    reg_table[f"{activity}_pro"] = reg_table[f"{activity}_labs"].map(lambda x:x[1] if str(x)!="nan" else 0)

reg_table.drop(columns=["channel_id_time", "channel_id_x", "channel_id_y", "view_labs"], inplace=True)


In [11]:
def bin_variable(val, var_range):
    out = 0
    if not np.isnan(val):
        if val <= var_range[1] and val >= var_range[0]:
            out = 1
    return out

reg_table["if_left"] = reg_table["lftRght"].map(lambda x: bin_variable(x, [1,3]))
reg_table["if_center"] = reg_table["lftRght"].map(lambda x: bin_variable(x, [4,4]))
reg_table["if_right"] = reg_table["lftRght"].map(lambda x: bin_variable(x, [5,7]))
reg_table["if_antigov"] = reg_table["feel_thm_fdsz"].map(lambda x: bin_variable(x, [0,2]))
reg_table["if_neutral"] = reg_table["feel_thm_fdsz"].map(lambda x: bin_variable(x, [3,7]))
reg_table["if_progov"] = reg_table["feel_thm_fdsz"].map(lambda x: bin_variable(x, [8,10]))

In [12]:
reg_table.to_csv("../../output/reg_table_nonfilter_channel_catg_counts.csv", index=False)